In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
import nest_asyncio

nest_asyncio.apply()

In [3]:
import pandas as pd
import os
import pathlib

root_dir = pathlib.PurePath(os.path.dirname(os.getcwd())).parent
data_dir = os.path.join(root_dir, 'data')

In [4]:
%pip install 'AutoRAG[gpu]'


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


# RAG 최적화 실습 with AutoRAG

## 전체 최적화 실행 (AutoRAG)

먼저 전체 최적화를 한 번에 실행한 후, 각 단계의 최적화 과정에 대해 더욱 자세히 알아봅시다.

In [5]:
sample_data_dir = os.path.join(data_dir, "optimization_sample")
qa_path = os.path.join(sample_data_dir, "qa.parquet")
corpus_path = os.path.join(sample_data_dir, "corpus.parquet")

In [6]:
# 데이터 확인
qa_df = pd.read_parquet(qa_path, engine='pyarrow')
corpus_df = pd.read_parquet(corpus_path, engine='pyarrow')

In [7]:
qa_df.head()

,qid,query,generation_gt,retrieval_gt
0,80_law,"원고 A이 시력 저하등의 증상이 발생한 시점은 언제로 보고, 그 시점 판단의 기준은...","[원고 A의 시력 저하 등의 증상은 2017년 2월 25일 이전, 즉 필러 주입술 ...",[[law - 민사_미성년자인.pdf - 7]]
1,4_finance,2024년도 한국은행의 대정부 일시대출금 한도와 대출조건을 결정하는 과정에서 보완된...,"[2024년도 한국은행의 대정부 일시대출금 한도는 총 50조 원으로, 이전 년도와 ...",[[finance - 2024년 3월_2. 통화신용정책 운영.pdf - 7]]
2,40_public,"2023년에 발표된 '가을철 지역축제 안전관리협조 요청 사항'은 무엇인지, 그리고 ...","[2023년 가을철 지역축제 안전관리협조 요청 사항에 따르면, 인파밀집 행사 시 부...",[[public - 국가안전시스템 개편 보고서.pdf - 8]]
3,69_law,국세기본법 제14조 제2항에서 언급하는 과세표준의 계산에 관한 규정이 어떠한 실질내...,"[국세기본법 제14조 제2항에 따르면, 과세표준의 계산은 소득, 수익, 재산, 행위...",[[law - 행정_원고가.pdf - 22]]
4,10_finance,"비은행 금융기관의 부동산 PF 대출 연체율 상승에 따른 미래적 영향은 무엇이며, 어...","[비은행 금융기관의 부동산 PF 대출의 연체율이 상승하는 경우, 이는 관련 대출 부...",[[finance - 2024년 3월_3. 향후 통화신용정책 방향.pdf - 10]]


In [8]:
len(qa_df)

15

실제 최적화에서는 50~100개 이상의 QA Pair를 사용하는 것을 권장합니다.

In [9]:
corpus_df.head()

,doc_id,contents,metadata
0,law - 형사_피고인이.pdf - 1,서울고등법원\n제3형사부\n판 결\n사 건 2023노1373 통신비밀보호법위반\n피...,{'last_modified_datetime': 2024-06-05 20:59:48...
1,law - 행정_원고가.pdf - 6,로열티로 지급하기로 하는 내용의 라이선스 계약을 체결하였다(갑 제4호증의 2). 그...,{'last_modified_datetime': 2024-06-05 20:59:48...
2,commerce - 이커머스 솔루션 소개자료.pdf - 36,e-Commerce Solution BESPIN GLOBAL\n#인앱영상통화 # 글...,{'last_modified_datetime': 2024-06-05 20:59:48...
3,law - 행정_타인자금.pdf - 5,소유주식수\n주주명 지분율 비고\n계 보통주 우선주\nc 33.730.000 33....,{'last_modified_datetime': 2024-06-05 20:59:48...
4,law - 행정_금품.pdf - 23,별지\n관계 법령\n공무원연금법\n제37조(급여의 환수)\n1 공단은 급여를 받은 ...,{'last_modified_datetime': 2024-06-05 20:59:48...


In [10]:
from autorag.evaluator import Evaluator

evaluator = Evaluator(
	qa_data_path=qa_path,
	corpus_data_path=corpus_path,
	project_dir=os.path.join(root_dir, "autorag_project", "evaluation", "optimization_sample"),
)

[01/23/25 21:02:59] INFO     [config.py:58] >> PyTorch version 2.5.1 available.                        ]8;id=295034;file:///Users/jeffrey/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/datasets/config.py\config.py]8;;\:]8;id=367123;file:///Users/jeffrey/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/datasets/config.py#58\58]8;;\

/Users/jeffrey/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_id" in DeployedModel has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/Users/jeffrey/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in HuggingFaceLLM has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/Users/jeffrey/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_kwargs" in HuggingFaceLLM has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/Users/jeffrey/PycharmProject

In [ ]:
evaluator.start_trial(yaml_path=os.path.join(root_dir, "config", "optimization", "config.yaml"), skip_validation=True)

이제 각 단계별 결과를 확인하며, 최적화 과정에 대해서 설명합니다.